In [8]:
import requests
import json

def fetch_stock_market_news(api_key, query):
    url = f"https://newsapi.org/v2/everything?q={query}&apiKey={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['articles']
    else:
        print("Failed to fetch news:", response.status_code)
        return None

def create_json_output(articles):
    output = []
    for article in articles:
        output.append({
            "title": article['title'],
            "date": article['publishedAt'],
            "source": article['source']['name'],
            "text": article['content']
        })
    return output

def main():
    api_key = "2afef13f6f4b4166aea1e34dd70d6083"
    query = "nvidia stock"
    articles = fetch_stock_market_news(api_key, query)
    if articles:
        output = create_json_output(articles)
        with open('nvidia_stock_news.json', 'w') as f:
            json.dump(output, f, indent=4)
        print("JSON file created successfully.")
    else:
        print("No articles fetched.")

if __name__ == "__main__":
    main()


JSON file created successfully.


In [9]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Join tokens back into text
    processed_text = ' '.join(tokens)
    return processed_text

def main():
    # Load Nvidia stock market articles from JSON file
    with open('nvidia_stock_news.json', 'r') as f:
        data = json.load(f)
    
    # Preprocess each article
    preprocessed_articles = []
    for article in data:
        processed_text = preprocess_text(article['text'])
        preprocessed_articles.append({
            "title": article['title'],
            "date": article['date'],
            "source": article['source'],
            "text": processed_text
        })

    # Save preprocessed articles to a new JSON file
    with open('preprocessed_nvidia_stock_news.json', 'w') as f:
        json.dump(preprocessed_articles, f, indent=4)

    print("Preprocessing complete. Preprocessed data saved to 'preprocessed_nvidia_stock_news.json'.")

if __name__ == "__main__":
    main()


Preprocessing complete. Preprocessed data saved to 'preprocessed_nvidia_stock_news.json'.


In [ ]:
# Import necessary libraries
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import Dataset, DataLoader
import json

# Define a custom dataset class
class NewsDataset(Dataset):
    def __init__(self, articles, tokenizer, max_length):
        self.articles = articles
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = self.articles[idx]['text']
        inputs = self.tokenizer.encode(article, add_special_tokens=True, max_length=self.max_length, truncation=True, padding='max_length')
        return torch.tensor(inputs)

# Load pre-trained tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Add padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))

# Check if a GPU is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Load preprocessed articles
with open('preprocessed_nvidia_stock_news.json', 'r') as f:
    preprocessed_articles = json.load(f)

# Define hyperparameters
batch_size = 4
max_length = 512
learning_rate = 1e-4
num_epochs = 5

# Prepare data loaders
dataset = NewsDataset(preprocessed_articles, tokenizer, max_length)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        inputs = batch.to(device)
        outputs = model(inputs, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss}")

# Save trained model
model.save_pretrained('trained_nvidia_model')
tokenizer.save_pretrained('trained_nvidia_model')

print("Model training complete and saved.")


In [ ]:
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

class AmericanOptionsLSMC:
    """American options pricing using Longstaff-Schwartz:
    """
    
    def __init__(self, option_type, S0, strike, T, M, r, div, sigma, simulations):
        if option_type not in ['call', 'put']:
            raise ValueError("Error: option type not valid. Enter 'call' or 'put'")
        if any(param < 0 for param in [S0, strike, T, r, div, sigma, simulations]):
            raise ValueError('Error: Negative inputs not allowed')
        
        self.option_type = option_type
        self.S0 = float(S0)
        self.strike = float(strike)
        self.T = float(T)
        self.M = int(M)
        self.r = float(r)
        self.div = float(div)
        self.sigma = float(sigma)
        self.simulations = int(simulations)
        
        self.time_unit = self.T / float(self.M)
        self.discount = np.exp((self.r - self.div) * self.time_unit)
        
        # Load the fine-tuned model and tokenizer for sentiment analysis
        self.tokenizer = GPT2Tokenizer.from_pretrained('trained_nvidia_model')
        self.model = GPT2LMHeadModel.from_pretrained('trained_nvidia_model')
        self.model.eval()

    def get_MCprice_matrix(self, seed=123):
        """Returns MC price matrix rows: time columns: price-path simulation"""
        np.random.seed(seed)
        MCprice_matrix = np.zeros((self.M + 1, self.simulations), dtype=np.float64)
        MCprice_matrix[0, :] = self.S0
        for t in range(1, self.M + 1):
            brownian = np.random.standard_normal(self.simulations // 2)
            brownian = np.concatenate((brownian, -brownian))
            MCprice_matrix[t, :] = (MCprice_matrix[t - 1, :] *
                                    np.exp((self.r - self.div - self.sigma ** 2 / 2.) * self.time_unit +
                                           self.sigma * brownian * np.sqrt(self.time_unit)))
        return MCprice_matrix
    
    @property
    def MCprice_matrix(self):
        return self.get_MCprice_matrix()

    @property
    def MCpayoff(self):
        """Returns the inner-value of American Option"""
        if self.option_type == 'call':
            payoff = np.maximum(self.MCprice_matrix - self.strike, 0)
        else:
            payoff = np.maximum(self.strike - self.MCprice_matrix, 0)
        return payoff

    @property
    def value_vector(self):
        value_matrix = np.zeros_like(self.MCpayoff)
        value_matrix[-1, :] = self.MCpayoff[-1, :]
        for t in range(self.M - 1, 0, -1):
            regression = np.polyfit(self.MCprice_matrix[t, :], value_matrix[t + 1, :] * self.discount, 5)
            continuation_value = np.polyval(regression, self.MCprice_matrix[t, :])
            value_matrix[t, :] = np.where(self.MCpayoff[t, :] > continuation_value,
                                          self.MCpayoff[t, :],
                                          value_matrix[t + 1, :] * self.discount)
        return value_matrix[1, :] * self.discount

    @property
    def price(self):
        return np.sum(self.value_vector) / float(self.simulations)

    @property
    def delta(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)

    @property
    def gamma(self):
        diff = self.S0 * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0 + diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0 - diff, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
        return (myCall_1.delta - myCall_2.delta) / float(2. * diff)

    @property
    def vega(self):
        diff = self.sigma * 0.01
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma + diff, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma - diff, self.simulations)
        return (myCall_1.price - myCall_2.price) / float(2. * diff)

    @property
    def rho(self):
        diff = self.r * 0.01
        if (self.r - diff) < 0:
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r + diff, self.div, self.sigma, self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r, self.div, self.sigma, self.simulations)
            return (myCall_1.price - myCall_2.price) / float(diff)
        else:
            myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r + diff, self.div, self.sigma, self.simulations)
            myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T, self.M, self.r - diff, self.div, self.sigma, self.simulations)
            return (myCall_1.price - myCall_2.price) / float(2. * diff)

    @property
    def theta(self):
        diff = 1 / 252.
        myCall_1 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T + diff, self.M, self.r, self.div, self.sigma, self.simulations)
        myCall_2 = AmericanOptionsLSMC(self.option_type, self.S0, self.strike, self.T - diff, self.M, self.r, self.div, self.sigma, self.simulations)
        return (myCall_2.price - myCall_1.price) / float(2. * diff)

    def generate_sentiment(self, article):
        inputs = self.tokenizer.encode(article, return_tensors='pt', max_length=512, truncation=True)
        outputs = self.model.generate(inputs, max_length=512, num_return_sequences=1)
        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Simplistic sentiment analysis based on keywords
        if 'positive' in generated_text:
            return 'positive'
        elif 'negative' in generated_text:
            return 'negative'
        else:
            return 'neutral'

    def adjust_parameters(self, sentiment):
        if sentiment == 'positive':
            self.S0 *= 1.05
            self.sigma *= 1.05
        elif sentiment == 'negative':
            self.S0 *= 0.95
            self.sigma *= 1.05

def main():
    # Example usage
    article = "Nvidia shares closed up 5% to $1,224.40 on Wednesday, giving the company a market cap above $3 trillion for the first time as investors continue to clamor for a piece of the company at the heart of the boom in generative artificial intelligence. Nvidia also passed Apple to become the second-largest public company behind Microsoft. Nvidia’s milestone is the latest stunning mark in a run that has seen the stock soar more than 3,224% over the past five years. The company will split its stock 10-for-1 later this month. Apple was the first U.S. company to reach a $3 trillion market cap during intraday trading in January 2022. Microsoft hit $3 trillion in market value in January 2024. Nvidia, which was founded in 1993, passed the $2 trillion valuation in February, and it only took roughly three months from there for it to pass $3 trillion. In May, Nvidia reported first-quarter earnings that showed demand for the company’s pricey and powerful graphics processing units, or GPUs, showed no sign of a slowdown. Nvidia reported overall sales of $26 billion, more than triple what it generated a year ago. Nvidia also beat Wall Street expectations for sales and earnings and said it would report revenue of about $28 billion in the current quarter. Nvidia’s surge in recent years has been powered by the tech industry’s need for its chips, which are used to develop and deploy big AI models such as the one at the heart of OpenAI’s ChatGPT. Companies such as Google, Microsoft, Meta, Amazon and OpenAI are buying billions of dollars worth of Nvidia’s GPUs." # input any news article.
    option_type = 'put'
    S0 = 36.
    strike = 40.
    T = 1.
    M = 50
    r = 0.06
    div = 0.06
    sigma = 0.2
    simulations = 10000

    american_option = AmericanOptionsLSMC(option_type, S0, strike, T, M, r, div, sigma, simulations)
    
    sentiment = american_option.generate_sentiment(article)
    print(f"Generated Sentiment: {sentiment}")

    american_option.adjust_parameters(sentiment)
    print(f"Adjusted S0: {american_option.S0}, Adjusted Sigma: {american_option.sigma}")
    print(f"Option Price: {american_option.price}")

if __name__ == "__main__":
    main()
